In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

/opt/tljh/user/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# Necessary metadata
article_id = 14226968  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "/srv/data/my_shared_data_folder/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'allyears.csv.zip',
  'supplied_md5': '9e046ac05ecd2c32a256a47dd1098b81',
  'computed_md5': '9e046ac05ecd2c32a256a47dd1098b81',
  'id': 26844650,
  'download_url': 'https://ndownloader.figshare.com/files/26844650',
  'size': 2405908113},
 {'is_link_only': False,
  'name': 'individual_years.zip',
  'supplied_md5': '921da748974b07b2a70bbfcc04535a77',
  'computed_md5': '921da748974b07b2a70bbfcc04535a77',
  'id': 26863682,
  'download_url': 'https://ndownloader.figshare.com/files/26863682',
  'size': 1896206676},
 {'is_link_only': False,
  'name': 'combined_model_data.csv.zip',
  'supplied_md5': '7638434c44a7d29cbb29fe200b4fd65d',
  'computed_md5': '7638434c44a7d29cbb29fe200b4fd65d',
  'id': 27515426,
  'download_url': 'https://ndownloader.figshare.com/files/27515426',
  'size': 821308997},
 {'is_link_only': False,
  'name': 'combined_model_data_parti.parquet.zip',
  'supplied_md5': '02f4e3df8d16580a02291de225072689',
  'computed_md5': '02f4e3df8d16580a02

In [4]:
files_to_dl = ["combined_model_data_parti.parquet.zip"]  ## Please download the partitioned 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [5]:
with zipfile.ZipFile(os.path.join(output_directory, "combined_model_data_parti.parquet.zip"), 'r') as f:
    f.extractall(output_directory)

### Data prep

In [11]:
df = pd.read_parquet("s3://mds-s3-student71/combined_model_data_parti.parquet")

In [12]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [13]:
syd_lat = -33.86
syd_lon = 151.21

In [14]:
transformed = df.query(f"lat_min < {syd_lat} and lat_max > {syd_lat} and lon_min < {syd_lon} and lon_max > {syd_lon}")\
    [["time", "model", "rain (mm/day)"]]
transformed['time'] = transformed["time"].dt.date

In [15]:
transformed = transformed.pivot(index="time", columns="model", values="rain (mm/day)")

In [16]:
df_rainfall = pd.read_csv("s3://mds-s3-student71/observed_daily_rainfall_SYD.csv", parse_dates=True, index_col="time")
df_rainfall["observed_rainfall"] = df_rainfall["rain (mm/day)"]

In [29]:
transformed = transformed.merge(df_rainfall[["observed_rainfall"]], left_index=True, right_index=True)

In [31]:
transformed.to_csv("s3://mds-s3-student71/output/ml_data_SYD.csv")